# 手工初级练丹

使用经典nmist数据集，训练数字识别的模型

* 没有使用keras的封装
* 用tf给的函数和数据类型构建一个3层的全链接网络

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets

## 准备数据集

这个地方还是用了一个Keras封装，自动处理：下载nmist的数据集，读入内存，转成tensor数据格式，分成训练和测试两部分。其实自己手写也不是很难。有时间手工实现一下。

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='./nmist.npz')

y_train = tf.one_hot(y_train, depth=10)

#print(x_train.shape, y_train.shape, y_train.dtype, x_test.shape, y_test.shape, y_test.dtype)

为了后面的测试数据对比的简便，这里就不对测试集中的输出真值做one_hot了

In [3]:
#y_test = tf.one_hot(y_test, depth=10)

## 创建数据包Dataset

对齐好的训练/测试数据TensorSliceDataset。原因很简单: <br/>

x_train [60k, 28, 28] y_train [60k,] 捆绑后，利于后期对数据的整体同步修改。 


In [4]:
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))



传递一个callback函数进入Dataset的map函数里，对数据的值域和类型做改变 <br/>
这里的def preprocess就是把训练集中x的原有int32转换到0.0-1.0之间的float32， y值改成了int64

In [5]:
def preprocess(x, y):
    #x = 1. + tf.cast(x, tf.float32) / 255.0 - 2 # [-1., 1.]
    x = tf.cast(x, tf.float32) / 255.0 # [0. , 1.]
    y = tf.cast(y, dtype=tf.int64)
    return x, y


* 打乱 -> shuffle
* 做批处理包 -> batch

In [6]:
db_train = db_train.map(preprocess).shuffle(60000).batch(100) 
db_test = db_test.map(preprocess).shuffle(10000).batch(100)


网络的向前传播 y = w*x + b，所需要的参数w: 节点链接的权重, b: bias误差参数，这里定义了一个三层的全连接网; 所有可以更新的参数，如w，b必须用tf.Variable封装下，以便被全局梯度函数监视

In [7]:
w1 = tf.Variable(tf.random.truncated_normal([28*28, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))

w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))

w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))


学习率，非常有用，是解决梯度爆炸和迷散的关键因素。一般来说用一个小点的开始练丹。不行了上，下调一下。

In [8]:
lr = 1e-3


## 练丹开始

epoch: 纪元，一组数据该练多少次
step: 第二个for里的这个step是根据上边定义batch的时候自动算出的，比如60K张图batch(10K),那一个纪元就得分6个steps搞完

*先把输入的图像张量里的数据轴(2,3两轴)敲成1维的
*正向传播了，就是乘法，加法然后激活函数
*算出的结果和真值做损失计算，这里使用了分类交叉熵函数，因为，mnist就是一个解决分类问题的模型
*用损失函数的结果计算坡度
*反向传播，使用梯度更新所有的网络参数，注意这里一定要使用assign_sub不然，参数的对象会不停的被重新创建，这样tape就不能监视值的变动了

这里尝试了MSE(mean squared error)均值平方差，和CCE(categorical crossentropy)分类交叉熵作为损失函数. 在训练的时候表现了明显的差别。<br/>
CCE收敛速度明显更快，准确度提升的也很快，基本3-5纪元就可以到达95%的准确度。总体50个纪元差不多能到98%. 而MSE 50个纪元只能到达80%左右。

每个纪元完毕后，检查一次作业，根据准确数和测试样本数确认准确率

In [9]:
for epoch in range(50):
    # Training...
    for step, (x, y) in enumerate(db_train):
        x = tf.reshape(x, [-1, 28*28])
        
        with tf.GradientTape() as tape:
            h1 = tf.nn.relu(x@w1 + b1)
            h2 = tf.nn.relu(h1@w2 + b2)
            out = h2@w3 + b3
            loss = tf.losses.categorical_crossentropy(y, out, from_logits=True)
            #loss = tf.reduce_mean(tf.square(tf.cast(y, dtype=tf.float32) - out))
        
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        
        w1.assign_sub(lr * grads[0])
        b1.assign_sub(lr * grads[1])
        
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])
        
        if step % 100 == 0:
            print('epoch:', epoch+1, 'batch:', step+1, 'loss: ', float(tf.reduce_mean(loss)))

    # Testing...
    total_correct, total_num = 0, 0
    for step, (x, y) in enumerate(db_test):
        x = tf.reshape(x, [-1, 28*28])
        
        h1 = tf.nn.relu(x@w1 + b1)
        h2 = tf.nn.relu(h1@w2 + b2)
        out = h2@w3 + b3

        prob = tf.nn.softmax(out, axis=1)
        pred = tf.argmax(prob, axis=1)
        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)
        
        total_correct += int(correct)
        total_num += x.shape[0]

    print('test acc:', total_correct * 100 / total_num, '%')

epoch: 1 batch: 1 loss:  2.4292261600494385
epoch: 1 batch: 101 loss:  0.540456235408783
epoch: 1 batch: 201 loss:  0.2897457778453827
epoch: 1 batch: 301 loss:  0.2927491366863251
epoch: 1 batch: 401 loss:  0.24325443804264069
epoch: 1 batch: 501 loss:  0.3704439103603363
test acc: 94.1 %
epoch: 2 batch: 1 loss:  0.1627996414899826
epoch: 2 batch: 101 loss:  0.15087324380874634
epoch: 2 batch: 201 loss:  0.12699632346630096
epoch: 2 batch: 301 loss:  0.13553215563297272
epoch: 2 batch: 401 loss:  0.20415328443050385
epoch: 2 batch: 501 loss:  0.20830899477005005
test acc: 95.48 %
epoch: 3 batch: 1 loss:  0.1231527253985405
epoch: 3 batch: 101 loss:  0.10673525929450989
epoch: 3 batch: 201 loss:  0.07322080433368683
epoch: 3 batch: 301 loss:  0.09598022699356079
epoch: 3 batch: 401 loss:  0.16377310454845428
epoch: 3 batch: 501 loss:  0.09559107571840286
test acc: 96.09 %
epoch: 4 batch: 1 loss:  0.10920644551515579
epoch: 4 batch: 101 loss:  0.08848544955253601
epoch: 4 batch: 201 los

KeyboardInterrupt: 